<a href="https://colab.research.google.com/github/anish-sk/CS6910_Assignment1/blob/master/src/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade wandb
!wandb login 6746f968d95eb71e281d6c7772a0469574430408

Requirement already up-to-date: wandb in /usr/local/lib/python3.6/dist-packages (0.10.19)
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Init wandb
import wandb

wandb.init(project="assignment1", entity="abisheks")

import numpy as np
from matplotlib import pyplot as plt
# Loading the fashion mnist dataset
from keras.datasets import fashion_mnist

# Setting seed value
np.random.seed(1)

# Load dataset (train data and test data)
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

# Summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

_runtime,11
_timestamp,1613821478
_step,0


_runtime,▁
_timestamp,▁
_step,▁


Train: X=(60000, 28, 28), y=(60000,)
Test: X=(10000, 28, 28), y=(10000,)


In [ ]:
# Number of classes in the Fashion-MNIST dataset
N_CLASSES = np.unique(trainy).shape[0]    # 10 as known from the keras documentation

# Captions/Labels for the output classes present in Fashion-MNIST dataset
IMG_LABELS = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


def getSampleImages(nClass, imgLabels, X, y, nSamples):
  '''
  The function takes few samples of each class from the dataset provided and passes it to the WANDB for it log the images

  Arguments :
    nClass - Number of output classes in the dataset
    imgLabels - List of labels for the output classes (numbered from 0 to nClass - 1)
    X - The input data containing images in the form of matrices
    y - The output data containing the class to which an input belongs
    nSamples - Number of samples of each class to be taken. If that many samples not present in dataset, maximum number of samples present (from that class) will be taken

  Returns :
    -- None --
  '''

  # Initialise empty list to store the input data sampled from each class
  sampleImgsX = [[] for _ in range(nClass)]

  # Take 3 sample images from each class
  for i in range(y.shape[0]):
    if len(sampleImgsX[y[i]]) < nSamples :
      sampleImgsX[y[i]].append(X[i])


  # Getting a list of sample images of each class to be saved to wandb
  sampleImgsList = []
  for i in range(nClass):
    for j in range(3):
      sampleImgsList.append(wandb.Image(sampleImgsX[i][j], caption = imgLabels[i]))

  np.random.shuffle(sampleImgsList)
  wandb.log({"example" : sampleImgsList})


# Question 1 : Show 3 sample images from training set of downloaded Fashion-MNIST dataset in WANDB
getSampleImages(N_CLASSES, IMG_LABELS, trainX, trainy, 3)

In [ ]:
def initialize_network(n_L):
  network = list()
  for i in range(1,L+1):
    layer = {'weights':np.random.randn(n_L[i], n_L[i-1])*0.01,
             'biases':np.zeros((n_L[i],1))}
    network.append(layer)	
  return network

wandb.config.update({"n_hidden_layers": 2, "size_hidden_layer":4})
n_L = [wandb.config['size_hidden_layer']] * (wandb.config['n_hidden_layers']+1)
L = wandb.config['n_hidden_layers']
n_L[0] = trainX.shape[1] * trainX.shape[2]
n_L[L] = N_CLASSES
network = initialize_network(n_L)
print(network)

[{'weights': array([[-0.00902068,  0.00503892, -0.00451881, ..., -0.01304595,
         0.01097441, -0.00370579],
       [-0.00639497, -0.01653854,  0.00676438, ..., -0.01586505,
        -0.01091546,  0.00674534],
       [ 0.02498861,  0.00570323,  0.02206509, ..., -0.01214146,
         0.00572396,  0.01017221],
       [ 0.01287594, -0.0125087 ,  0.00381013, ...,  0.00259758,
        -0.00219102, -0.00741698]]), 'biases': array([[0.],
       [0.],
       [0.],
       [0.]])}, {'weights': array([[ 0.00502532, -0.01612423,  0.00604293, -0.0016596 ],
       [ 0.01387487, -0.00650048,  0.00179392, -0.00808011],
       [-0.00072224,  0.01511209, -0.00314805, -0.00647256],
       [ 0.00659218, -0.0081968 ,  0.00830372, -0.00728754],
       [-0.00253982, -0.01769889,  0.00879207, -0.00076894],
       [-0.00210317,  0.01151913, -0.00992107,  0.00944247],
       [ 0.00338278, -0.01597519,  0.00051305, -0.00159809],
       [-0.01585   , -0.00143523,  0.00671293, -0.00151243],
       [-0.00160107,

In [ ]:
def sigmoid(x):
  return np.exp(-np.logaddexp(0, -x))

def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum()

def linear(W, X, b):
  return W @ X + b

def pre_activation(H_prev, W, b, pre_activation_fn):
  print(W.shape, H_prev.shape, b.shape)
  A = linear(W, H_prev, b)
  

  assert(A.shape == (W.shape[0], H_prev.shape[1]))
  cache = (H_prev, W, b)

  return A, cache

def activation(H_prev, W, b, activation_fn, pre_activation_fn):

  A, pre_activation_cache = pre_activation(H_prev, W, b, pre_activation_fn)
  H = activation_fn(A)
    
  assert (H.shape == (W.shape[0], H_prev.shape[1]))
  cache = (pre_activation_cache)

  return A, cache

In [ ]:
def feedforward(X, network, hidden_activation_fn, output_activation_fn, hidden_pre_activation_fn, output_pre_activation_fn):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    H = X
    L = len(network)              # number of layers in the neural network
    
    for l in range(0, L-1):
        H_prev = H 
        H, cache = activation(H_prev, network[l]['weights'], network[l]['biases'], hidden_activation_fn, hidden_pre_activation_fn)
        caches.append(cache)
    
    HL, cache = activation(H, network[L-1]['weights'], network[L-1]['biases'], output_activation_fn, output_pre_activation_fn)
  
    caches.append(cache)
    
    assert(HL.shape == (N_CLASSES, X.shape[1]))
            
    return HL, caches

HL, caches = feedforward(trainX.reshape((trainX.shape[1]*trainX.shape[2], trainX.shape[0])), network, sigmoid, softmax, linear, linear)
print(HL)

(4, 784) (784, 60000) (4, 1)
(10, 4) (4, 60000) (10, 1)
[[ 0.00333972 -0.01317929 -0.21668674 ... -0.14399805 -0.36399945
  -0.45987612]
 [ 0.31574531  0.03394593 -0.27990774 ...  0.16025583 -0.05012484
  -0.12118353]
 [ 0.07344705 -0.01275872  0.08259799 ...  0.28743918  0.39312906
   0.32445701]
 ...
 [-0.49222701 -0.14637818  0.10644949 ... -0.14374338 -0.11595683
  -0.38129861]
 [-0.00131982 -0.42473172 -0.86671804 ...  0.02175913  0.23869986
   0.64957134]
 [-0.03792192  0.1667716   0.4421885  ... -0.02271715  0.02247633
  -0.02426223]]


(60000, 28, 28)